In [1]:
# pip install scipy

In [2]:
%load_ext autoreload
%autoreload 2
import torch
from training_utils import train_model, test
from models import BBandFChead

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/me.docker/.conda/envs/torch-p37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn

resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
# resnet = resnet50(pretrained=True, ) 
resnet.fc = nn.Identity() # removing pretrained last layer
resnet_linear = BBandFChead(resnet, hidden_dim=0, output_dim=101) # creating a model with frozen bb and only a linear eval head.

/home/me.docker/.conda/envs/torch-p37/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jovyan/.conda/envs/torch-p37/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")


In [4]:
from torch.utils.data import DataLoader
from caltech101 import load_caltech101

train_dataset, test_dataset = load_caltech101(data_path="/home/me.docker/work/data")
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=1)


6941 1736


In [ ]:
resnet_linear = BBandFChead(resnet, hidden_dim=0, output_dim=101) # creating a model with frozen bb and only a linear eval head.
train_model(resnet_linear, train_dataset=train_dataset, test_dataset=test_dataset, batch_size=16,
            lr=0.0001, epochs=5, samples_per_epoch=None, test_max_samples=None)

Train; Epoch 1, Step 434: Average loss: 0.2196, Accuracy: 1979/6941 (28.51%), TPR: 35.54%, TNR: 47.81%, TNR+TPR/2: 41.67%
Test; Epoch 1, Step 6941 (18.7s): Average loss: 2.9067, Accuracy: 818/1736 (47.12%), TPR: 67.57%, TNR: 58.18%, TNR+TPR/2: 62.87%
Train; Epoch 2, Step 868: Average loss: 0.1457, Accuracy: 4196/6941 (60.45%), TPR: 75.12%, TNR: 73.15%, TNR+TPR/2: 74.13%
Test; Epoch 2, Step 13882 (16.2s): Average loss: 2.2095, Accuracy: 1263/1736 (72.75%), TPR: 92.00%, TNR: 84.82%, TNR+TPR/2: 88.41%
Train; Epoch 3, Step 1302: Average loss: 0.1060, Accuracy: 5704/6941 (82.18%), TPR: 92.10%, TNR: 88.37%, TNR+TPR/2: 90.23%
Test; Epoch 3, Step 20823 (16.5s): Average loss: 1.7687, Accuracy: 1437/1736 (82.78%), TPR: 97.22%, TNR: 89.62%, TNR+TPR/2: 93.42%
Train; Epoch 4, Step 1736: Average loss: 0.0798, Accuracy: 6287/6941 (90.58%), TPR: 94.75%, TNR: 94.48%, TNR+TPR/2: 94.61%
Test; Epoch 4, Step 27764 (16.5s): Average loss: 1.4627, Accuracy: 1505/1736 (86.69%), TPR: 100.00%, TNR: 92.31%, TNR+T

In [ ]:
from matplotlib import pyplot as plt
f, axes = plt.subplots(3,3, figsize=(15,15))
mult = len(train_dataset)//9
for i, ax in enumerate(axes.flatten()):
    img = train_dataset[i*mult][0].permute(1,2,0)
    ax.imshow(img)
    ax.title.set_text(train_dataset[i*mult][1])

In [ ]:
img.min()